# 1. Подготовка данных

Импортируем необходимые библиотеки, функции и структуры данных.

In [1]:
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from tqdm import notebook
tqdm.pandas()


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Сохраним в переменной data входные данные с комментариями пользователей интернет-магазина "Викишоп".

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
display(data.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Создадим функцию clear_text для обработки текста: лемматизации, удаления символов, пробелов. Добавим в таблицу данных столбец с обработанными комментариями пользователей. Выведем на экран таблицу данных.

In [4]:
def clear_text(text):
    token_list = nltk.word_tokenize(text)
    token_text = ' '.join(token_list)
    text_row = re.sub(r'[^a-zA-Z]', ' ', token_text)
    clear_text = ' '.join(text_row.split())
    return clear_text

In [5]:
notebook.tqdm.pandas() 

In [6]:
data['clear_text'] = data['text'].progress_apply(clear_text)

display(data.head())

,text,toxic,clear_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n t make any real suggestions on imp...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


Определим соотношение позитивных и негативных комментариев пользователей.

In [7]:
class_frequency = data['toxic'].value_counts(normalize=True)
print(class_frequency)

0    0.898321
1    0.101679
Name: toxic, dtype: float64


Наблюдается существенный дисбаланс классов в целевом признаке.

Разделим данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Создадим переменные features_train, features_valid и features_test с признаком - лемматизированные комментарии и целеdst признаки target_train, target_valid, target_test с целевым признаком - позитивный или негативный комментарий.

In [8]:
train, valid_test = train_test_split(data, test_size = 0.4, random_state=12345)
valid, test = train_test_split(valid_test, test_size = 0.5, random_state=12345)

In [9]:
features_train = train['clear_text']
target_train = train['toxic']

features_valid = valid['clear_text']
target_valid = valid['toxic']

features_test = test['clear_text']
target_test = test['toxic']

Выведем размеры выборок после разделения.

In [10]:
print(features_train.shape)
print(target_train.shape)

print(features_valid.shape)
print(target_valid.shape)

print(features_test.shape)
print(target_test.shape)

(95742,)
(95742,)
(31914,)
(31914,)
(31915,)
(31915,)


Создадим корпус для обучающей, валидационной и тестовой выборок.

In [11]:
corpus_train = features_train.values.astype('U')
corpus_valid = features_valid.values.astype('U')
corpus_test = features_test.values.astype('U')

Загрузим стоп-слова и очистим лемматизированный текст. Создадим счетчик TfidfVectorizer.

In [12]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf_train = TfidfVectorizer(stop_words=stopwords) 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Рассчитаем значение TF-IDF для корпуса текстов обучающей, валидационной и тестовой выборок. На обучающей выборке загрузим функцию fit(). Выведем размер матрицы TF-IDF для выборок.

In [13]:
tf_idf_train = count_tf_idf_train.fit_transform(corpus_train)
print("Размер матрицы:", tf_idf_train.shape)

Размер матрицы: (95742, 125587)


In [14]:
tf_idf_valid = count_tf_idf_train.transform(corpus_valid)
print("Размер матрицы:", tf_idf_valid.shape)

Размер матрицы: (31914, 125587)


In [15]:
tf_idf_test = count_tf_idf_train.transform(corpus_test)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (31915, 125587)


# 2. Обучение

Обучим модели логистической регрессии и решающего дерева для классификации токсичности комментариев. Поскольку в целевом признаке наблюдается существенный дисбаланс классов, в аргументе моделей укажем параметр class_weight='balanced'. Рассчитаем значение f1-меры для каждой модели на валидационной выборке.

#### 2.1 Логистическая регрессия

In [16]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(tf_idf_train, target_train)

predicted_valid = model.predict(tf_idf_valid)

print('F1-мера:', f1_score(target_valid, predicted_valid))

F1-мера: 0.7536032944406315


#### 2.2 Модель решающего дерева

In [17]:
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model.fit(tf_idf_train, target_train)

predicted_valid = model.predict(tf_idf_valid)

print('F1-мера:', f1_score(target_valid, predicted_valid))

F1-мера: 0.6450287592440428


# 3. Выводы

Рассчитаем значение F1-меры для модели логистической регрессии на тестовой выборке.

In [18]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(tf_idf_train, target_train)

predicted_test = model.predict(tf_idf_test)

print('F1-мера:', f1_score(target_test, predicted_test))

F1-мера: 0.7533545441969843


На основе данных с разметкой о токсичности правок построена модель для классификации характера комментариев пользователей интернет-магазина "Викишоп". Обучена модель логистической регрессии с учетом дисбаланса классов. Значение метрика качества F1 предсказаний на тестовой выборке составляет **~0.75**, что удовлетворяет требованиям, учтановленным в условиях задачи.